In [2]:
import torch
import numpy as np
from sklearn import metrics
import torch.nn as nn
from scipy import sparse
from torch.utils.data import DataLoader,Dataset

In [ ]:
def load_data():
    dise_sim = np.load('data\disease_disease.npy')#138*138
    c_d = np.load('data\circRNA_disease.npy')#834*138
    c_m = np.load('data\circRNA_miRNA.npy')#834*555
    m_d = np.load('data\disease_miRNA.npy').T #555*138
    return torch.tensor(dise_sim,dtype=float),torch.tensor(c_d,dtype=torch.long),torch.tensor(c_m,dtype=torch.long),torch.tensor(m_d,dtype=torch.long)

import itertools
def calculate_sim(cd,dd):
    s1=cd.shape[0]
    ll=torch.eye(s1)
    m2=dd*cd[:,None,:]
    m1=cd[:,:,None]
    for x,y in itertools.permutations(torch.linspace(0,s1-1,s1,dtype=torch.long),2):
        x,y=x.item(),y.item()
        m=m1[x,:,:]*m2[y,:,:]
        if cd[x].sum()+cd[y].sum()==0:
            ll[x,y]=0
        else:
            ll[x,y]=(m.max(dim=0,keepdim=True)[0].sum()+m.max(dim=1,keepdim=True)[0].sum())/(cd[x].sum()+cd[y].sum())
    return ll

In [ ]:
#n就是n折交叉
def split_dataset(n,c_d,dise_sim):
    #正样本，找出所有坐标
    cd_axix = torch.where(c_d==1) #989   表示989对关联
    circ_x = cd_axix[0] #989
    dise_y = cd_axix[1] #989
    #有顺序的坐标
    positive_sample = torch.cat([circ_x[:,None],dise_y[:,None]],dim=1) #989*2
    rand_index1 = torch.randperm(c_d.sum()) #0~988
    #打乱顺序
    ps = torch.index_select(positive_sample,dim=0,index=rand_index1) #989*2，在上面的positive_sample中按列随机选中
    
    #负样本
    cd_axix_neg = torch.where(c_d==0) 
    circ_x_neg = cd_axix_neg[0]  #114103
    dise_y_neg = cd_axix_neg[1]  #114103
    #有顺序的坐标
    negative_sample = torch.cat([circ_x_neg[:,None],dise_y_neg[:,None]],dim=1) #114103*2
    rand_index2 = torch.randperm(negative_sample.shape[0])
    ns = torch.index_select(input=negative_sample,dim=0,index=rand_index2) #114103*2    
    #开始5折交叉验证
    p_len = ps.shape[0]
    one_sample_len = int(p_len/n) #197
    train_dataset,test_dataset,cd_cover,circ_sims =[],[],[],[]
    #5倍交叉，可以理解为5次不同的训练集和测试集，分为4个训练集一个测试集，989*0.8=788,
    #如果有1k个，第一次就选0-800，第二次200-1000？
    for i in range(n):
        #训练集 每轮取四fold
        train_p = torch.cat([ps[:i*one_sample_len,:],ps[(i+1)*one_sample_len:n*one_sample_len,:]],dim=0).T #788
        train_n = torch.cat([ns[:i*one_sample_len,:],ns[(i+1)*one_sample_len:n*one_sample_len,:]],dim=0).T #788
        train_data = torch.cat([train_p,train_n],dim=1)# 2*1576
        train_dataset.append(train_data)
        #测试集 取剩下的一fold
        test_p = torch.cat((ps[i*one_sample_len:(i+1)*one_sample_len,:],ps[n*one_sample_len:,:]),dim=0).T #2*197+4
        test_n = torch.cat((ns[i*one_sample_len:(i+1)*one_sample_len,:],ns[n*one_sample_len:,:]),dim=0).T #2*(197+113118)

        test_data = torch.cat([test_p,test_n],dim=1)
        test_dataset.append(test_data)
        #进行遮盖
        cover_one = c_d.clone()
        row_one = test_p[0,:]
        row_two = test_p[1,:]
        cover_one[row_one,row_two]=0 #将测试集的正例置0 
        print('坐标')
        print(test_p[:,0])
        print('遮盖，这里应该为false')
        print(cover_one[test_p[0,0],test_p[1,0]])
        print(c_d[test_p[0,0],test_p[1,0]])
        cd_cover.append(cover_one)
        
        c_sim = calculate_sim(cover_one,dise_sim)
        circ_sims.append(c_sim)
        
    return train_dataset,test_dataset,cd_cover,circ_sims
dise_sim,c_d,c_m,m_d = load_data()
train_dataset,test_dataset,cd_cover,circ_sims = split_dataset(n=5,c_d=c_d,dise_sim=dise_sim)
#构造大的cover矩阵
mi_sim = calculate_sim(m_d,dise_sim)

circ_sim =circ_sims
print(circ_sim[0].shape)
print(circ_sim[0])
def make_cover_matrix(n,circ_sim,cd_cover,c_m,dise_sim,m_d,mi_sim):
    #获取数据

    #五倍交叉，构造五次
    cover_feature_matrix = []
    for i in range(n):
        circ_row = torch.cat([circ_sim[i],cd_cover[i],c_m],dim=1)
        dise_row = torch.cat([cd_cover[i].T,dise_sim,m_d.T],dim=1)
        mi_row = torch.cat([c_m.T,m_d,mi_sim],dim=1)
        cover_matrix = torch.cat([circ_row,dise_row,mi_row],dim=0)
        cover_feature_matrix.append(cover_matrix)
    return cover_feature_matrix

cover_feature_matrix = make_cover_matrix(5,circ_sim,cd_cover,c_m,dise_sim,m_d,mi_sim)
torch.save(c_d,R'data\circ_disease.pth')
torch.save(cd_cover,R'data\cd_cover.pth')
# print(cover_feature_matrix[0].shape)
# print(cover_feature_matrix[1].shape)
# print(cover_feature_matrix[2].shape)
# print(cover_feature_matrix[3].shape)
# print(cover_feature_matrix[4].shape)
torch.save(train_dataset,R'data\train_dataset.pth')
torch.save(test_dataset,R'data\test_data.pth')
torch.save(cover_feature_matrix,R'data\cover_feature_matrix.pth')

In [ ]:
adj_matrix_list = torch.load(rf'data\cover_feature_matrix.pth')
cd,ccc,cdc,dcd,ddd,cc,dd = [],[],[],[],[],[],[]
for i in range(5):
    adj_matrix = adj_matrix_list[i]
    circ_sim = adj_matrix[0:834,0:834]
    c_d = adj_matrix[0:834,834:834+138]

    di_sim = adj_matrix[834:834+138,834:834+138]
    
    c_c_c = torch.matmul(circ_sim,circ_sim)
    c_d_c = torch.matmul(c_d,c_d.T)
    
    d_c_d = torch.matmul(c_d.T,c_d)
    d_d_d = torch.matmul(di_sim,di_sim)
    
    # meta_c = torch.cat([c_d,c_c_c,c_d_c],dim=1)
    # meta_d = torch.cat([c_d.T,d_d_d,d_c_d],dim=1)

    cd.append(c_d)
    ccc.append(c_c_c)
    cdc.append(c_d_c)
    dcd.append(d_c_d)
    ddd.append(d_d_d)
    cc.append(circ_sim)
    dd.append(di_sim)
torch.save(cd,rf'data\meta_path\cd.pth')
torch.save(cc,rf'data\meta_path\cc.pth')
torch.save(dd,rf'data\meta_path\dd.pth')
torch.save(ccc,rf'data\meta_path\ccc.pth')
torch.save(cdc,rf'data\meta_path\cdc.pth')
torch.save(dcd,rf'data\meta_path\dcd.pth')
torch.save(ddd,rf'data\meta_path\ddd.pth')
